In [1]:
import pyro
import torch
import ssl
import matplotlib.pyplot as plt


from src.logger.logger_initializer import LoggerInitializer
from src.models.bnn.pyro_miniresnet_bnn_classifier import PyroMiniresnetBnnClassifier
from src.utils.path_getter import PathGetter
from src.data_loader.dataset_getter import DatasetGetter


ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__


LoggerInitializer().init()
dataset = DatasetGetter(PathGetter.get_data_directory(), batch_size=12).get()

[2023-04-17 21:45:28] 5929 root {logger_initializer-47} INFO - Current process ID: 5929


In [3]:
classifier = PyroMiniresnetBnnClassifier(dataset=dataset, device=device).init()

In [ ]:
classifier.unfreeze_all_layers()
num_epoch = 300
result_df, result_dict = classifier.fit(num_epoch=num_epoch)
classifier.save_model(result_dict)
result_df.to_csv(f'{classifier.name}_{classifier._uuid}_{num_epoch}_epoch.csv')
# display(result_df)
# print(result_dict)

[2023-04-17 21:47:08] 5929 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 0/300]: f1_score: 0.378 accuracy_score: 0.424 loss: 489363120638261.250 
[2023-04-17 21:47:22] 5929 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 1/300]: f1_score: 0.347 accuracy_score: 0.387 loss: 370466100446058.125 
[2023-04-17 21:47:36] 5929 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 2/300]: f1_score: 0.356 accuracy_score: 0.402 loss: 272595562417152.875 
[2023-04-17 21:47:49] 5929 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 3/300]: f1_score: 0.384 accuracy_score: 0.418 loss: 193705863551448.656 
[2023-04-17 21:48:03] 5929 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 4/300]: f1_score: 0.370 accuracy_score: 0.406 loss: 131748756249869.266 
[2023-04-17 21:48:24] 5929 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 5/300]: f1_score: 0.379 accuracy_score: 0.434 l

In [ ]:
classifier.score(dataset.validate_dataloader)

In [ ]:
result_df.to_csv(f'{classifier.name}_300_epoch_freeze_all_layers_except_fc_scores.csv')

In [ ]:
test_model_weights_path = '../../bayesian_refactoring/assets/model_weights/pyro_resnet18bnn_28766_97cd2c3b-14db-49e6-9035-0474b00c51be.pt'

In [ ]:
test_results = torch.load(test_model_weights_path)

In [ ]:
new_classifier = PyroMiniresnetBnnClassifier(
    dataset=dataset, device=device
).load_model(
    model_state_dict=test_results['best_f1_score_model'],
    pyro_state_dict=test_results['pyro_state_dict']
)

In [ ]:
new_classifier.score(dataset.validate_dataloader)